# This Model is trained using Retinanet

REFERENCE_LINK = https://github.com/fizyr/keras-retinanet

About retinanet :Keras implementation of RetinaNet object detection as described in Focal Loss for Dense Object Detection by Tsung-Yi Lin, Priya Goyal, Ross Girshick, Kaiming He and Piotr Dollár.

#  Importing Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

>  ## INSTALLING  RETINANET

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
!pip install --upgrade keras

In [ ]:
%cd keras-retinanet/

!pip install .

In [ ]:
!python setup.py build_ext --inplace


In [ ]:
import os
import tensorflow as tf
import cv2
import tempfile
import numpy as np 
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:

from tensorflow import keras
import seaborn as sns
from pylab import rcParams
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
import csv
import cv2
import time
from sklearn.model_selection import train_test_split
#RETINANET LIBRARIES
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline

%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10


## Data PreProcessing

In [ ]:
%cd ..
csv_pth = '../input/face-mask-detection-dataset/train.csv'

df = pd.read_csv(csv_pth,header=None)
df = df.iloc[1:]
df[0] = '../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/' + df[0].astype(str)

print(df.head())


print("==============================================================================")

ndf = pd.read_csv(csv_pth,index_col='name')

dict1={}
categories=set(ndf['classname'])
for j,i in enumerate(categories):
    dict1[i] = j+1
print(dict1)




In [ ]:
cordinates=list(ndf.iloc[:,0:4].values)

ndf['cordinates'] = cordinates
ndf=ndf.drop(['x1', 'x2','y1','y2'], axis = 1) 
print(ndf)

In [ ]:
src_dir = r'../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/'
src_img=os.listdir(src_dir)
test_src=src_img[:1698]
train_src=src_img[1698:]

print(len(train_src))
print(len(test_src))

train_img=train_src
train_img_path=[os.path.join(src_dir, fname) for fname in train_img]

test_img=test_src
test_img_path=[os.path.join(src_dir, fname) for fname in test_img]

In [ ]:
import matplotlib.patches as patches

def bounds_and_taggs(img):
    coords=[];classes=[]
    ndf1=(ndf.loc[[img]].values)
    #print(ndf1)
    for i in ndf1:
        classes.append(i[0])
        coords.append(i[1])
    classes= list(set(classes))
    return coords,classes

## PRE_TRAINING

In [ ]:
train_df, test_df = train_test_split(
  df, 
  test_size=0.15, 
  shuffle=False
)

In [ ]:
train_df.iloc[8131:8132,:]
train_df=train_df.drop([8132], axis=0)

In [ ]:
print("trining set len ->",len(train_df))
print("testing set len ->",len(test_df))

In [ ]:
train_annots_file = 'train_annots.csv'
test_annots_file = 'test_annots.csv'
classes_file='classes.csv'

In [ ]:
train_df.to_csv(train_annots_file, index=False, header=None)
test_df.to_csv(test_annots_file, index=False, header=None)

In [ ]:
!head train_annots.csv
#!head test_annots.csv

In [ ]:
labels = dict([(value, key) for key, value in dict1.items()])
labels


In [ ]:
with open(classes_file, 'w') as f:
  for i,j in labels.items():
    f.write('{},{}\n'.format(j,i))

In [ ]:
!head classes.csv
!tail classes.csv

## Training the Model

In [ ]:
import urllib

PRETRAINED_MODEL = 'pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

In [ ]:
!pip install tensorflow-gpu

In [ ]:
ANNOTATIONS_FILE = 'train_annots.csv'
CLASSES_FILE = 'classes.csv'

In [ ]:
!keras-retinanet/keras_retinanet/bin/train.py \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--batch-size 8 \
--steps 100 \
--epochs 5 \
csv train_annots.csv classes.csv


In [ ]:
!ls snapshots

In [ ]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])

print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

## Making Predictions

In [ ]:
labels_to_names = pd.read_csv(classes_file, header=None).T.loc[0].to_dict()
labels_to_names

In [ ]:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels

In [ ]:
THRES_SCORE = 0.55

def draw_detections(image, boxes, scores, labels):
    
    coordinates=[];category=[];

    for box, score, label in zip(boxes[0], scores[0], labels[0]):

        if score < THRES_SCORE:
            break

        color = label_color(label)
        #print(label)

        b = box.astype(int)
        draw_box(image, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label-1], score)
        draw_caption(image, b, caption)
        category.append(labels_to_names[label-1])
        coordinates.append(box)

    return coordinates,category
    

In [ ]:

def show_detected_objects(image_row):
    img_path = image_row["name"]
#     img_dir='../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images'
#     img_path=os.path.join(img_dir, img_path)

    image = read_image_bgr(img_path)

    boxes, scores, labels = predict(image)

    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    true_box = [
      image_row["x1"], image_row["x2"], image_row["y1"], image_row["y2"]
    ]
    draw_box(draw, true_box, color=(255, 255, 0))
    coordinates,category=draw_detections(draw, boxes, scores, labels)
    final_df=pd.DataFrame(coordinates,columns=['x1' , 'x2', 'y1', 'y2'])
    final_df['label']=category
    final_df['name']=image_row['name']
    final_df=final_df[['name','x1' , 'x2', 'y1', 'y2','label']]
    return final_df



    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [ ]:
test_df.columns = ['name', 'x1', 'x2', 'y1','y2','classname']
test_df

In [ ]:
show_detected_objects(test_df.iloc[558])

## Generating Submission File

In [ ]:
submission=pd.read_csv("../input/face-mask-detection-dataset/submission.csv")
submission = submission.drop_duplicates()
submission.head()

In [ ]:
submission_df=pd.DataFrame(columns=['name','x1' , 'x2', 'y1', 'y2','label'])

In [ ]:
for i in range(0,len(submission)):
    b=show_detected_objects(submission.iloc[i])
    submission_df=submission_df.append(b,ignore_index = True)
submission_df.head()

In [ ]:
submission_df

In [ ]:
complete=submission_df.to_csv(r'submission_csv.csv')